In [1]:
import group_data as gd
import flylib
import numpy as np

In [2]:
%pylab inline
%config InlineBackend.figure_format = 'svg'
import pylab as plb
plb.rcParams['pdf.fonttype'] = 42

Populating the interactive namespace from numpy and matplotlib


### Calculate the cross covariance data for each signal type and add it to the trial_db

In [3]:
#fit to each fly in serial (block) but break up the data to run in parallel within a fly.
from IPython.parallel import Client
clients = Client() 
clients.block = True
print clients.ids
v = clients[:]

IOError: Connection file u'~/.ipython/profile_nbserver/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [ ]:
#amplitude
import h5py
import numpy as np
trial_db = h5py.File('trial_db.hdf5','a')

muscle_list =  ['b1','b2','b3',
                'i1','i2',
                'iii1','iii3','iii24',
                'hg1','hg2','hg3','hg4',
                'tpd','tpv','ttm','pr']
from scipy.signal import correlate
from scipy.signal import detrend
corr_dict = dict()

for mname in muscle_list:
    sarray1 = np.array(trial_db[mname])
    sarray2 = np.array(trial_db['Ph1'])
    xdata = [detrend(a1[np.isfinite(a1)]) for a1 in sarray1]
    ydata = [detrend(a1[np.isfinite(a1)]) for a1 in sarray2]
    corrlist = v.map(correlate,xdata,ydata,['same']*len(xdata))
    corrmtrx = np.empty((len(corrlist),1149))
    corrmtrx[:] = np.nan
    for i,cd in enumerate(corrlist):
        if np.shape(cd)[0] > 1147:
            corrmtrx[i,:1147] = cd[:1147]
        else:
            pass
    corr_dict[mname] = corrmtrx
for key in corr_dict:
    trial_db.create_dataset('xcov_amp' + key,data = np.array(corr_dict[key]),  compression="gzip",compression_opts=5)
trial_db.flush()

In [ ]:
#frequency
import h5py
import numpy as np
trial_db = h5py.File('trial_db.hdf5','a')

muscle_list =  ['b1','b2','b3',
                'i1','i2',
                'iii1','iii3','iii24',
                'hg1','hg2','hg3','hg4',
                'tpd','tpv','ttm','pr']
from scipy.signal import correlate
from scipy.signal import detrend
corr_dict = dict()

def wrapped_correlate(xdata,ydata,mode):
    from scipy.signal import correlate
    import numpy as np
    if np.squeeze(np.shape(xdata)) == np.squeeze(np.shape(ydata)):
        return correlate(xdata,ydata,mode)
    else:
        return np.nan
     
for mname in muscle_list:
    sarray1 = np.array(trial_db[mname])
    sarray2 = np.array(trial_db['wb_frequency'])
    xdata = [detrend(a1[np.isfinite(a1)]) for a1 in sarray1]
    ydata = [detrend(a1[np.isfinite(a1)]) for a1 in sarray2]
    corrlist = v.map(wrapped_correlate,xdata,ydata,['same']*len(xdata))
    corrmtrx = np.empty((len(corrlist),1149))
    corrmtrx[:] = np.nan
    for i,cd in enumerate(corrlist):
        if np.squeeze(np.shape(cd)) > 1147:
            corrmtrx[i,:1147] = cd[:1147]
        else:
            pass
    corr_dict[mname] = corrmtrx
for key in corr_dict:
    trial_db.create_dataset('xcov_freq' + key,data = np.array(corr_dict[key]),  compression="gzip",compression_opts=5)
trial_db.flush()

### summarize xcov data accross flies - just stripe fixation for now.

In [3]:
import h5py
trial_db = h5py.File('trial_db.hdf5','r')
xi = np.linspace(0,23,1150)
tlag = xi-xi[1150/2.0]
tlag = tlag[:1149]
muscle_list =  ['b1','b2','b3',
                'i1','i2',
                'iii1','iii3','iii24',
                'hg1','hg2','hg3','hg4',
                'tpd','tpv','ttm','pr']
swarm_names = np.array(trial_db['swarm_names'])
fly_numbers = np.array(trial_db['fly_numbers'])
wb_frequency = np.array(trial_db['wb_frequency'])
trial_names = np.array(trial_db['trial_names'])
wb_selection = np.sum(wb_frequency >50,axis = 1)>1000
swarm_set = [sw for sw in set(swarm_names) if not('pr' in sw)]
fly_set = set(fly_numbers)

/home/psilentp/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:15: RuntimeWarning: invalid value encountered in greater


In [4]:
#calculate the median xcov functions between stroke amplitude and muscle activity
#load into a dictionary keyed by group and then muscle
cov_dict = dict()
trial_selection = (trial_names == 'stripe_fix')
wb_selection = np.sum(wb_frequency >50)>1000
wb_selection &= trial_selection
for swarm in swarm_set:
    print swarm
    swarm_selection = (swarm == swarm_names) & wb_selection
    cov_dict[swarm] = dict()
    for mname in muscle_list:
        cov_dict[swarm][mname] = list()
        for flynum in fly_set:
            selection  = (flynum == fly_numbers) & swarm_selection
            if sum(selection) > 2:
                trace = nanmedian(np.array(trial_db['xcov_amp%s'%(mname)])[selection,:],axis = 0)
                #trace = np.nanmean(np.array(trial_db['xcov_amp%s'%(mname)])[selection,:],axis = 0)
                #trace = nanmedian(np.array(trial_db['xcov_amp_ms%s'%(mname)])[selection,:],axis = 0)
                cov_dict[swarm][mname].append(trace)                
#xcov_amp_ms

/home/psilentp/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: RuntimeWarning: invalid value encountered in greater
/home/psilentp/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:662: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


GMR22H05_GFP
GMR31E10
GMR75B06_GFP
GMR22H05
GMR29E05_GFP
GMR39E01
GMR31E10_GFP
GMR10A12
GMR74F03_GFP
GMR39E01_GFP
GMR75B06
GMR74F03
GMR29E05
GMR10A12_GFP


In [5]:
import h5py
import os
#os.listdir('./')
os.remove('cov_db_amp_sf.hdf5')
cov_db = h5py.File('cov_db_amp_sf.hdf5','a')
for key in swarm_set:
    cov_db.create_group(key)
    for key2 in cov_dict[key].keys():
        cov_db[key].create_dataset(key2,data = np.array(cov_dict[key][key2]),  compression="gzip",compression_opts=5)
cov_db.flush()

In [1]:
#calculate the median xcov functions between stroke frequency and muscle activity
#load into a dictionary keyed by group and then muscle
cov_dict_freq = dict()
trial_selection = (trial_names == 'stripe_fix')
wb_selection = np.sum(wb_frequency >50)>1000
wb_selection &= trial_selection
for swarm in swarm_set:
    print swarm
    swarm_selection = (swarm == swarm_names) & wb_selection
    cov_dict_freq[swarm] = dict()
    for mname in muscle_list:
        cov_dict_freq[swarm][mname] = list()
        for flynum in fly_set:
            selection  = (flynum == fly_numbers) & swarm_selection
            if sum(selection) > 2:
                trace = nanmedian(np.array(trial_db['xcov_freq%s'%(mname)])[selection,:],axis = 0)
                cov_dict_freq[swarm][mname].append(trace)

NameError: name 'trial_names' is not defined

In [6]:
import h5py
import os
#os.listdir('./')
#os.remove('cov_db_freq_sf.hdf5')
cov_db_freq = h5py.File('cov_db_freq_sf.hdf5','a')
for key in swarm_set:
    cov_db_freq.create_group(key)
    for key2 in cov_dict_freq[key].keys():
        cov_db_freq[key].create_dataset(key2,data = np.array(cov_dict_freq[key][key2]),  compression="gzip",compression_opts=5)
cov_db_freq.flush()